In [1]:
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GridSearchCV
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from xgboost import XGBRegressor
import numpy as np

In [2]:
df_raw = pd.read_excel("realized.xlsx", engine='openpyxl')
df_raw.shape

(39449, 9)

In [3]:
df_raw = df_raw.dropna(how='any')
df_raw = df_raw[df_raw['LoadFactor'] != 0]
df_raw.shape

(39389, 9)

In [4]:
df_raw.head()

,ScheduleTime,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,SeatCapacity,LoadFactor
0,2021-01-01 06:35:00,IA,874,DEN,73W,J,US,142,0.408451
1,2021-01-01 10:35:00,JZ,818,YHM,AT7,J,CA,74,0.189189
2,2021-01-01 12:05:00,IA,876,DEN,73W,J,US,142,0.570423
3,2021-01-01 13:20:00,CN,514,EST,AT7,J,US,72,0.333333
4,2021-01-01 14:20:00,LJ,3140,DEN,32A,J,US,186,0.204301


In [5]:
def make_time_features(df):
    for feature in ["Airline", "Destination", "AircraftType", "FlightType", "Sector"]:
        df[feature] = df[feature].astype(str).astype('category')

    df['Minute'] = df['ScheduleTime'].dt.minute
    df['Hour'] = df['ScheduleTime'].dt.hour
    df['Day'] = df['ScheduleTime'].dt.day
    df['Week'] = df['ScheduleTime'].dt.isocalendar().week.astype(int)
    df['Month'] = df['ScheduleTime'].dt.month
    season_mapping = {12: "Winter", 1: "Winter", 2: "Winter",
                    3: "Spring", 4: "Spring", 5: "Spring",
                    6: "Summer", 7: "Summer", 8: "Summer",
                    9: "Autumn", 10: "Autumn", 11: "Autumn"}
    df['Season'] = df['Month'].apply(lambda x: season_mapping[x])
    return df
# df = make_time_features(df)

def convert_time_to_unixtime(df):
    df['ScheduleTime'] = (df['ScheduleTime'] - pd.Timestamp("1970-01-01"))// pd.Timedelta('1s')
    return df

df = convert_time_to_unixtime(df_raw)

In [6]:
def categorical_to_dummies(df):
    return pd.get_dummies(df, columns=["FlightNumber", "Airline", "Destination", "AircraftType", "FlightType", "Sector"])


In [7]:
def threshold_df(df, threshold):
    var_thresh = VarianceThreshold(threshold=threshold).fit(df)
    thres_column_names = df.columns[var_thresh.get_support()]
    X = var_thresh.transform(df)
    return X, thres_column_names, var_thresh

In [8]:
def flight_accuracy(LoadFactor_true, LoadFactor_hat, SeatCapacity):
    actual_passangers = LoadFactor_true*SeatCapacity
    forecasted_passangers = LoadFactor_hat*SeatCapacity
    deviation_per_flight = (actual_passangers - forecasted_passangers) / (actual_passangers)
    abs_deviation_per_flight = np.abs(deviation_per_flight)
    abs_deviation_per_flight[abs_deviation_per_flight >= 10000] = 100

    mean_forecast_acc = np.mean(1 - abs_deviation_per_flight*1)*100
    return mean_forecast_acc

# Splitting the data
Since normal splitting stratigies assumes that data is i.i.d and we can clearly see that there are trends in the data we'll use `TimeSeriesSplit`

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

def perform_splits(df, threshold, n_estimators, criterion, bootstrap, max_leaf_nodes, max_depth):
    df = categorical_to_dummies(df)
    X, thres_column_names, _ = threshold_df(df.drop(['LoadFactor'], axis=1), threshold)
    y = df['LoadFactor'].to_numpy()

    n = 5
    tscv = TimeSeriesSplit(n_splits=n)

    flight_acc_list = []
    r2_score_list = []
    for i, (train_index, test_index) in enumerate(tscv.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train = StandardScaler().fit_transform(X_train, y_train)
        X_test = StandardScaler().fit_transform(X_test, y_test)

        RF_regr = RandomForestRegressor(n_estimators=n_estimators,
                                        criterion=criterion, 
                                        bootstrap=bootstrap, 
                                        max_leaf_nodes=max_leaf_nodes, 
                                        max_depth=max_depth, oob_score=True, n_jobs=16)

        RF_regr.fit(X_train, y_train)
        
        y_hat = RF_regr.predict(X_test)
        score = r2_score(y_test, y_hat)
        r2_score_list.append(score)
        
        SeatCapacity_idx = np.argwhere(thres_column_names == "SeatCapacity")
        SeatCapacity = X_test[:,SeatCapacity_idx]

        flight_acc = flight_accuracy(y_test, y_hat, SeatCapacity)
        flight_acc_list.append(flight_acc)

        print(f"Score of split {i}: {round(score, 2)}")
        print(f"Flight acc of split {i}: {round(flight_acc, 2)}")
        
    mean_flight_acc = np.mean(flight_acc_list)
    mean_r2_score = np.mean(r2_score_list)
    return mean_flight_acc

In [10]:
df_eval = df_raw[int(39389*.8):]
df_train = df_raw[:int(39389*.8)]

In [11]:
import optuna

def objective(trial):
    criterion = "mse" # = trial.suggest_categorical('criterion', ['mse', 'mae'])
    bootstrap = True #  trial.suggest_categorical('bootstrap',['True','False'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 10000)
    threshold = trial.suggest_uniform("threshold", 0.0, 0.25)
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, 100)
    max_depth = None # = trial.suggest_int('max_depth', 1, 10000)
    return perform_splits(df_train, threshold, n_estimators, criterion, bootstrap, max_leaf_nodes, max_depth)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

[I 2022-03-21 20:08:55,121] A new study created in memory with name: no-name-864746db-59c4-40df-b4d3-c22cac7ae251


Score of split 0: -0.42
Flight acc of split 0: 25.35
Score of split 1: -0.16
Flight acc of split 1: 34.12
Score of split 2: -0.46
Flight acc of split 2: 23.49
Score of split 3: -0.35
Flight acc of split 3: -24.38


[I 2022-03-21 20:09:57,799] Trial 0 finished with value: 20.44473671754699 and parameters: {'max_leaf_nodes': 6846, 'threshold': 0.07892536051171423, 'n_estimators': 1700}. Best is trial 0 with value: 20.44473671754699.


Score of split 4: -0.47
Flight acc of split 4: 43.64
Score of split 0: -0.28
Flight acc of split 0: 31.81
Score of split 1: -0.01
Flight acc of split 1: 40.81
Score of split 2: -0.29
Flight acc of split 2: 29.91
Score of split 3: -0.32
Flight acc of split 3: -20.0


[I 2022-03-21 20:10:36,843] Trial 1 finished with value: 24.883613488759504 and parameters: {'max_leaf_nodes': 3807, 'threshold': 0.03001413622704613, 'n_estimators': 900}. Best is trial 1 with value: 24.883613488759504.


...

Score of split 4: -0.25
Flight acc of split 4: 43.95
Score of split 0: -0.48
Flight acc of split 0: 27.99
Score of split 1: -0.14
Flight acc of split 1: 37.36
Score of split 2: -0.42
Flight acc of split 2: 25.59
Score of split 3: -0.38
Flight acc of split 3: -22.16


[I 2022-03-21 20:17:55,480] Trial 3 finished with value: 22.097403041547732 and parameters: {'max_leaf_nodes': 6747, 'threshold': 0.05254705942531801, 'n_estimators': 2000}. Best is trial 2 with value: 34.55177419055461.


Score of split 4: -0.45
Flight acc of split 4: 41.7
Score of split 0: -0.43
Flight acc of split 0: 23.2
Score of split 1: -0.19
Flight acc of split 1: 34.76
Score of split 2: -0.43
Flight acc of split 2: 28.69
Score of split 3: -0.24
Flight acc of split 3: -18.12


[I 2022-03-21 20:18:16,687] Trial 4 finished with value: 22.338767124365084 and parameters: {'max_leaf_nodes': 7316, 'threshold': 0.16015154715167573, 'n_estimators': 400}. Best is trial 2 with value: 34.55177419055461.


Score of split 4: -0.44
Flight acc of split 4: 43.17
Score of split 0: -0.37
Flight acc of split 0: 40.14
Score of split 1: 0.03
Flight acc of split 1: 49.94
